# Teil 4 Demo 1: Abfragen mit SQL

In dieser Demo wird gezeigt, wie Medizinische Beispieldaten (aus der MIMIC III-Datenbank) mit SQL-Befehlen abgefragt werden können.

## Bibliotheken & Konfiguration

In diesem Abschnitt werden benötigte Programmpakete geladen und Konfigurationsvariablen z.B. für die Datenquellen gesetzt.

Hinweis: auf Google Colab kann das Laden der Pakete vor allem beim ersten Aufruf einige Minuten dauern. Bitte führen Sie diesen Block dann nicht erneut aus, sondern warten die Ausführung ab.

In [ ]:
packages <- c("readr", "DBI", "RSQLite")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-Datenanalyse-Visualisierung/master/Rohdaten/mimic-iii-demo/"

## Ausgewählte MIMIC III-Rohdaten laden

Die Rohdaten des MIMIC III-Datensatzes liegen in Form einzelner CSV-Dateien vor. Um sie mit SQL abfragen zu können, müssen sie zunächst in eine Datenbank geladen werden. Wir verwenden hierzu die SQLite-Datenbank, die sehr geringe Systemanforderungen hat und ohne einen dedizierten Datenbankserver betrieben werden kann.

Als ersten Schritt laden wir die CSV-Dateien in den Speicher und nutzen dazu die schon aus dem Open Data-Beispiel bekannte read_delim-Funktion und geben für die Spalten jeweils auch explizite Datentypen an.

In [ ]:
# Patientenstammdaten laden
mimic.patients.raw <- read_delim(paste0(base_url, "PATIENTS.csv"),
                                 col_types = cols(row_id = col_double(), subject_id = col_double(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()),
                                 skip = 0, delim = ",")
head(mimic.patients.raw)

In [ ]:
# Behandlungsfälle laden
mimic.admissions.raw <- read_delim(paste(base_url, "ADMISSIONS.csv", sep=""),
                                   col_types = cols(  row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()),
                                   skip = 0, delim = ",")
head(mimic.admissions.raw)

In [ ]:
# Verschreibungen laden
mimic.prescriptions.raw <- read_delim(paste(base_url, "PRESCRIPTIONS.csv", sep=""),
                                      col_types = cols(row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), icustay_id = col_double(), startdate = col_datetime(format = ""), enddate = col_datetime(format = ""), drug_type = col_character(), drug = col_character(), drug_name_poe = col_character(), drug_name_generic = col_character(), formulary_drug_cd = col_character(), gsn = col_character(), ndc = col_character(), prod_strength = col_character(), dose_val_rx = col_character(), dose_unit_rx = col_character(), form_val_disp = col_character(), form_unit_disp = col_character(), route = col_character()),
                                      skip = 0, delim = ",")
head(mimic.prescriptions.raw)

## Rohdaten in SQLite-Datenbank (im Speicher) laden

Im nächsten Schritt schreiben wir die 3 Datensätze in eine SQLite-Datenbank. Die `dbWriteTable`-Funktion legt hierbei automatisch eine Tabelle mit den Spaltennamen und Datentypen des zuvor gelesenen Dataframes an. Mit der `dbListTables()`-Funktion lassen wir uns abschließend eine Liste der angelegten Tabellen anzeigen.

In [ ]:
con <- dbConnect(RSQLite::SQLite(), ":memory:")
dbWriteTable(con, "patients",      mimic.patients.raw)
dbWriteTable(con, "admissions",    mimic.admissions.raw)
dbWriteTable(con, "prescriptions", mimic.prescriptions.raw)
dbListTables(con)

## Einfache Abfragen auf Rohdaten-Ebene

### Alle Spalten einer Tabelle abfragen

In dieser Zelle fragen wir einfach alle Zeilen und Spalten der `patienten`-Tabelle ab. Um das Notebook nicht unübersichtlich zu machen, zeigen wir mit der `head()`-Funktion nur die ersten 6 Zeilen der Ergebnismenge an.

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM patients
"))

### Ausgewählte Spalten einer Tabelle abfragen

Statt der Wildcard `*` können wir im `SELECT`-Statement auch einzelne Spalten angeben, die wir (statt allen Spalten zusammen) abfragen wollen:

In [ ]:
head(dbGetQuery(con, "
SELECT subject_id,
       gender
  FROM patients
"))

### Ausgewählte Zeilen einer Tabelle abfragen

Wenn wir nur einen Teil der Datensätze (Zeilen) der Tabelle abfragen wollen, können wir die `WHERE`-Klausel benutzen, um Selektionskriterien anzugeben (hier: Geschlecht männlich)

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM patients
 WHERE gender = 'M'
"))

### Ergebnis auf eindeutige Datensätze reduzieren (DISTINCT)

Wenn man sich nur für die eindeutigen Ausprägungen des Ergebnisdatensatzes (z.B. die Werteliste einer Spalte) interessiert, kann man das `DISTINCT`-Keyword an den Anfang des `SELECT`-Statements setzen. Doppelte Datensätze werden zusammengeführt, so dass nur noch die eindeutigen Ausprägungen ausgegeben werden (hier die Ausprägungen der Geschlecht-Spalte).

In [ ]:
head(dbGetQuery(con, "
SELECT DISTINCT
       gender
  FROM patients
"))

## Daten aggregieren

### Einfache Aggregation: Gesamtzahl der Datensätze einer Tabelle abfragen

Neben der Ausgabe von Einzeldatensätzen kann auch aggregiert werden. Die einfachste Form ist das Zählen der Gesamtanzahl von Datensätzen in der Treffermenge mit der `COUNT(*)`-Funktion.

In [ ]:
head(dbGetQuery(con, "
SELECT COUNT(*) AS n
  FROM patients
"))

### Aggregation & Gruppierung: Anzahl nach Geschlecht ermitteln

Häufig benötigt man aggregierte Angaben für bestimmte Gruppen von Datensätzen (hier z.B. die Patientenanzahl nach Geschlecht). Über die `GROUP BY`-Klausel können dabei Spalten angegeben werden, die als Gruppierungsmerkmal genutzt werden sollen. Diese Spalten werden typischerweise identisch im SELECT-Statement verwendet, damit sie zusammen mit der aggregierten Kennzahl ausgegeben werden können.

In [ ]:
head(dbGetQuery(con, "
SELECT gender,
       COUNT(*) AS n
  FROM patients
 GROUP BY gender
"))

### Zeilen nach der Aggregation filtern

Wir haben oben gesehen, wie mit der `WHERE`-Klausel Rohdatensätze anhand von Filterkriterien selektiert werden können. Manchmal ist es darüber hinaus nötig, Datensätze nach einer Aggregation zu filtern. Hierzu kann die `HAVING`-Klausel verwendet werden, die erst nach einer Aggregation angewendet wird. In diesem Beispiel aggregieren wir die Fallzahl nach Geschlecht, und filtern anschließend auf Ergebnisdatensätze, die in der aggregierten Patientenzahl mehr als 50 Einträge haben.

In [ ]:
head(dbGetQuery(con, "
SELECT gender,
       COUNT(*) AS n
  FROM patients
 GROUP BY gender
HAVING COUNT(*) > 50
"))

## Mehrere Tabellen miteinander verbinden

### Tabellen per JOIN miteinander verknüpfen

Relationale Datenbanken leben davon, dass Tabellen miteinander über Primär-/Fremdschlüsselbeziehungen verbunden werden können. In SQL wird die `JOIN`-Klausel verwendet, um zwei oder mehr Tabellen miteinander zu verknüpfen. Dabei muss immer mindestens ein Kriterium angegeben werden, das für die Verknüpfung der Zeilen beider Tabellen angewendet werden kann. Typischerweise wird dabei der Primärschlüssel der einen Tabelle (z.B. Patienten-ID der Patientenstammdatentabelle ) als Fremdschlüssel in der zweiten Tabelle (z.B. Behandlungsfälle mit Verknüpfung zu Patienten) verwendet.
Standardmäßig wird hierbei ein "INNER JOIN" durchgeführt, d.h. es werden nur die Datensätze ausgegeben, die in beiden Tabellen die JOIN-Kriterien erfüllen. Datensätze, zu denen es in der jeweils anderen Tabelle keine Entsprechung gibt, werden ausgefiltert (z.B. Patienten ohne Behandlungsfall).

Da Spalten zwischen beiden Tabellen identische Namen haben können, müssen für die einbezogenen Tabellen Aliasnamen vergeben werden (im Beispiel unten "pat" für die patients-Tabelle und "adm" für die admissions-Tabelle). Alle Spalten sollten mit dem jeweiligen Alias präfixiert werden, damit die Datenbank weiß, aus welcher Tabelle sie das jeweilige Datenelement nehmen soll.

In [ ]:
head(dbGetQuery(con, "
SELECT pat.subject_id,
       pat.gender,
       adm.diagnosis
  FROM patients   pat
  JOIN admissions adm ON pat.subject_id = adm.subject_id
"))

## Patient:innen mit mehr als einem Intensiv-Aufenthalt abfragen

Über die per JOIN verbundenen Tabellen können alle oben beschriebenen Funktionen angewendet werden, um z.B. per `WHERE`-Klausel Rohdatensätze zu filtern oder per `GROUP BY`/Aggregatfunktion Datensätze zu zählen oder aufzusummieren.

In [ ]:
head(dbGetQuery(con, "
SELECT pat.subject_id,
       COUNT(*) AS n_admissions
  FROM patients   pat
  JOIN admissions adm ON pat.subject_id = adm.subject_id
 GROUP BY pat.subject_id
HAVING COUNT(*) > 1
 ORDER BY COUNT(*) DESC
"))

## Komplexe Abfragen

## 2 Subsets von Verschreibungen für die Demo verschiedener Joins erzeugen

Für die folgenden Beispiele legen wir zwei Teilmengen von Patienten in zusätzlichen Tabellen an:
* Patient:innenen mit Hauptdiagnose Sepsis
* Patient:innen mit Gabe von Vancomycin (Reserve-Antibiotikum z.B. bei multiresistenten Staphylococcus Aureus-Infektionen)

In [ ]:
dbExecute(con, "CREATE TABLE demo_sepsis     AS SELECT DISTINCT subject_id FROM admissions    WHERE LOWER(diagnosis) LIKE '%sepsis%'")
dbExecute(con, "CREATE TABLE demo_vancomycin AS SELECT DISTINCT subject_id FROM prescriptions WHERE LOWER(drug)      LIKE '%vancomycin%'")

### Patient:innen abfragen, die sowohl eine Sepsis als Hauptdiagnose hatten als auch Vancomycin erhalten haben (INNER JOIN)

In diesem Beispiel fragen wir die Schnittmenge von Patienten ab, die sowohl eine Sepsis-Diagnose als auch eine Vancomycin-Diagnose hatten. Hierzu verwenden wir wieder einen INNER JOIN, wie im vorherigen Beispiel.

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM demo_sepsis     sep
  JOIN demo_vancomycin van on sep.subject_id = van.subject_id
"), 100)

Wir sehen als Output 6 Datensätze, jeweils mit der gleichen Patienten-ID auf der linken und rechten Seite. Es handelt sich hier wegen des INNER JOINs nur um die Patienten, die in beiden Tabellen (Sepsis & Vancomycin-Gabe) vorhanden sind.

### Alle Patient:innen abfragen, die eine Sepsis als Hauptdiagnose hatten sowie die mit Vancomycingabe ergänzen (LEFT OUTER JOIN)

Im nächsten Schritt wollen wir alle Patienten abfragen, die eine Sepsis hatten und uns zusätzlich ausgeben lassen, welche von Ihnen eine Vancomycin-Gabe hatten. Da wir hier einen "LEFT OUTER JOIN" verwenden, werden auch Datensätze ausgegeben, die keine Vancomycin-Gabe hatten. Bei diesen wird im rechten Teil "NA" (not available) ausgegeben.

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM      demo_sepsis     sep
  LEFT JOIN demo_vancomycin van on sep.subject_id = van.subject_id
"), 100)

In der Ausgabe sehen wir jetzt nicht mehr 6, sondern 8 Patienten. Bei den beiden neu hinzugekommenen Patienten ist in der rechten Spalte "NA" angegeben.

### Alle Patient:innen abfragen, die eine Vancomycingabe hatten und die keine Sepsisdiagnose hatten (RIGHT OUTER JOIN)

Neben dem LEFT OUTER JOIN gibt es auch den umgekehrten Fall des RIGHT OUTER JOINs. Hierbei werden alle Datensätze der rechten Tabelle beibehalten und nur die passenden Datensätze der linken Tabelle hinzugefügt.

In unserem Beispiel entspricht das allen Patienten, die eine Vancomycin-Gabe hatten, mit Anzeige der Patienten, die zusätzlich eine Sepsis dokumentiert bekommen haben.

**Hinweis**: SQLite unterstützt keine RIGHT JOINS, deshalb ist das hier als LEFT JOIN mit vertauschter Reihenfolge dargestellt

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM      demo_vancomycin van
  LEFT JOIN demo_sepsis     sep on van.subject_id = sep.subject_id
"), 100)

Wir shene im Output jetzt 55 Patienten, die Vancomycin erhalten haben. Hiervon ist bei 6 eine Patienten-ID in der rechten Spalte angegeben. Es handelt sich hierbei um die Patienten, die zusätzlich auch eine Sepsis dokumentiert bekommen haben.

### Tatsächliche Hauptdiagnosen für die Patient:innen mit Vancomycingabe ohne Sepsis ermitteln

Im letzten Schritt wollen wir die Hauptdiagnose der Patienten anzeigen, die  Vancomycin erhalten haben, aber keine Sepsis dokumentiert bekommen haben.

Wir filtern dazu auf Datensätze ohne Eintrag in der Sepsis-Spalte (`WHERE sep.subject_id IS NULL`) und ergänzen einen INNER JOIN auf die Behandlungsfall-Tabelle (admissions), in der die Hauptdiagnose angegeben ist. Wir aggregieren anschließend auf das Diagnosefeld und zählen die Anzahl der Datensätze.

In [ ]:
head(dbGetQuery(con, "
SELECT adm.diagnosis,
       COUNT(*)
  FROM      demo_vancomycin van
  JOIN      admissions      adm ON van.subject_id = adm.subject_id
  LEFT JOIN demo_sepsis     sep on van.subject_id = sep.subject_id
 WHERE sep.subject_id IS NULL
 GROUP BY adm.diagnosis
 ORDER BY COUNT(*) DESC
"), 10)

Wir sehen im Output, dass jede Diagnose nur relativ wenige Fallzahlen hat, aber Fieber die häufigste mit 3 Fällen ist.

## Datenbankverbindung schließen

In [ ]:
dbDisconnect(con)